# Sensor characterization plots (4–6 figures total)

Generates the characterization plots in your thesis plan.

Assumed CSV columns (matches your attached sample):  
`t, b1..b6, x, y, fx, fy, fz, tx, ty, tz`

Each section begins with a `DATA_PATH = r"..."` so you can point to a different dataset.


In [1]:
# ----------------- GLOBAL SETUP -----------------
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.colors import LinearSegmentedColormap
import re
import os

SENSOR_VERSION = "5.109"

# ---- Output directory for saving figures ----
OUTPUT_DIR = r"C:\Users\aurir\OneDrive - epfl.ch\Thesis- Biorobotics Lab\Sensor Characterization"
os.makedirs(OUTPUT_DIR, exist_ok=True)

# ---- Plot style ----
plt.rcParams["figure.dpi"] = 140
plt.rcParams["figure.facecolor"] = "white"
plt.rcParams["axes.facecolor"] = "#fafafa"
plt.rcParams["axes.edgecolor"] = "#cccccc"
plt.rcParams["axes.linewidth"] = 0.6
plt.rcParams["grid.linewidth"] = 0.3
plt.rcParams["grid.alpha"] = 0.4
plt.rcParams["grid.color"] = "#999999"
plt.rcParams["font.size"] = 11
plt.rcParams["axes.titlesize"] = 13
plt.rcParams["axes.titleweight"] = "bold"
plt.rcParams["axes.labelsize"] = 12
plt.rcParams["legend.fontsize"] = 10
plt.rcParams["legend.framealpha"] = 0.85
plt.rcParams["xtick.labelsize"] = 10
plt.rcParams["ytick.labelsize"] = 10

# Dark navy -> Teal -> Emerald -> Green -> Yellow palette
COLORS = ["#292f56", "#005c7f", "#008780", "#44b155", "#d6c52e", "#3a7a9e"]
CMAP_THEME = LinearSegmentedColormap.from_list(
    "thesis", ["#292f56", "#005c7f", "#008780", "#44b155", "#d6c52e"], N=256)

# Columns expected (based on your attached CSV)
BARO_COLS = ["b1","b2","b3","b4","b5","b6"]
FORCE_COLS = ["fx","fy","fz"]
POS_COLS   = ["x","y"]

def load_data(path):
    df = pd.read_csv(path)
    # Ensure numeric
    for c in BARO_COLS + FORCE_COLS + POS_COLS:
        if c in df.columns:
            df[c] = pd.to_numeric(df[c], errors="coerce")
    # Time normalization (optional)
    if "t" in df.columns:
        t0 = df["t"].iloc[0]
        df["t_s"] = df["t"] - t0
        # If t looks like ns, rescale
        if df["t_s"].median() > 1e6:
            df["t_s"] = (df["t"] - t0) * 1e-9
    else:
        df["t_s"] = np.arange(len(df))/100.0
    return df

def mean_pressure(df):
    return df[BARO_COLS].mean(axis=1)

def moving_average(x, win):
    if win <= 1:
        return x
    return pd.Series(x).rolling(win, center=True, min_periods=1).mean().to_numpy()

def segment_runs_by_contact(df, fz_col="fz", contact_thr=0.2, min_len=200):
    # Returns (start,end) segments where |Fz| >= contact_thr
    fz = df[fz_col].to_numpy()
    contact = np.abs(fz) >= contact_thr
    segs = []
    in_seg = False
    s = 0
    for i, c in enumerate(contact):
        if c and not in_seg:
            in_seg = True
            s = i
        if in_seg and (not c or i == len(contact)-1):
            e = i if not c else i+1
            if e - s >= min_len:
                segs.append((s,e))
            in_seg = False
    return segs

def pick_left_right_top_bottom_pairs():
    # Fill this once to match your PCB layout
    return dict(left="b1", right="b3", top="b2", bottom="b5")

## 1) Normal force response
Dataset: repeated ramps 0→10 N→0 at a fixed central location.

In [2]:
# ----------------- 1) NORMAL FORCE RESPONSE -----------------
DATA_PATH = rf"C:\Users\aurir\OneDrive - epfl.ch\Thesis- Biorobotics Lab\train_validation_test_data\train_data_v{SENSOR_VERSION}1.csv"

df = load_data(DATA_PATH)

# Optional smoothing
P = moving_average(mean_pressure(df), win=11)
Fz = moving_average(df["fz"].to_numpy(), win=11)

# If your Fz is negative in compression, flip it for readability
Fz_plot = -Fz

# Split into repeats (segments with contact)
segs = segment_runs_by_contact(df, fz_col="fz", contact_thr=0.2, min_len=30)
print(f"Found {len(segs)} contact segments")

# ---- Figure: overlay repeats, mean pressure vs Fz ----
plt.figure(figsize=(5.5,4))
max_runs = min(5, len(segs))
for k in range(max_runs):
    s,e = segs[k]
    plt.plot(Fz_plot[s:e], P[s:e], color=COLORS[k], label=f"run {k+1}", alpha=0.9, linewidth=1.5)
plt.xlabel("|Fz| [N]")
plt.ylabel("Mean pressure (b1..b6) [hPa]")
plt.title("Normal response: mean pressure vs Fz (repeats)")
if max_runs > 1:
    plt.legend()
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "01_normal_response_repeats.png"), dpi=300, bbox_inches='tight')
plt.close()

# ---- Figure: sensitivity slopes in low and high ranges ----
def slope_in_range(F, P, lo, hi):
    m = (F>=lo) & (F<=hi) & np.isfinite(F) & np.isfinite(P)
    if m.sum() < 5:
        return np.nan
    return np.polyfit(F[m], P[m], 1)[0]

slopes_0_2 = []
slopes_2_10 = []
for k in range(max_runs):
    s,e = segs[k]
    F = Fz_plot[s:e]
    Pr = P[s:e]
    slopes_0_2.append(slope_in_range(F, Pr, 0, 2))
    slopes_2_10.append(slope_in_range(F, Pr, 2, 10))

labels = [f"run {i+1}" for i in range(max_runs)]
x = np.arange(max_runs)

plt.figure(figsize=(6,3.5))
plt.bar(x-0.2, slopes_0_2, width=0.4, color=COLORS[0], label="0-2 N")
plt.bar(x+0.2, slopes_2_10, width=0.4, color=COLORS[2], label="2-10 N")
plt.xticks(x, labels)
plt.ylabel("Slope dP/dFz [hPa/N]")
plt.title("Normal sensitivity in low vs high force range")
plt.grid(True, axis="y")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "02_normal_sensitivity_ranges.png"), dpi=300, bbox_inches='tight')
plt.close()

print("Mean slope 0-2 N:", np.nanmean(slopes_0_2), "hPa/N")
print("Mean slope 2-10 N:", np.nanmean(slopes_2_10), "hPa/N")

Found 5 contact segments
Mean slope 0-2 N: 0.09925029186696957 hPa/N
Mean slope 2-10 N: 0.07044871092285485 hPa/N


In [3]:
# ---- Figure: Loading vs Unloading with hysteresis ----
all_Fz_load, all_P_load = [], []
all_Fz_unload, all_P_unload = [], []

for k in range(max_runs):
    s, e = segs[k]
    F_seg = Fz_plot[s:e]
    P_seg = P[s:e]
    peak_idx = np.argmax(F_seg)
    all_Fz_load.append(F_seg[:peak_idx+1])
    all_P_load.append(P_seg[:peak_idx+1])
    all_Fz_unload.append(F_seg[peak_idx:])
    all_P_unload.append(P_seg[peak_idx:])

Fz_load = np.concatenate(all_Fz_load)
P_load = np.concatenate(all_P_load)
Fz_unload = np.concatenate(all_Fz_unload)
P_unload = np.concatenate(all_P_unload)

fig, ax = plt.subplots(figsize=(7, 5))
ax.scatter(Fz_load, P_load, alpha=0.5, s=20, label='Loading', color=COLORS[2])
ax.scatter(Fz_unload, P_unload, alpha=0.5, s=20, label='Unloading', color=COLORS[3])

if len(Fz_load) > 10:
    z_load = np.polyfit(Fz_load, P_load, 2)
    p_load = np.poly1d(z_load)
    fz_range = np.linspace(Fz_load.min(), Fz_load.max(), 100)
    ax.plot(fz_range, p_load(fz_range), '--', color=COLORS[2], linewidth=2, alpha=0.7)

if len(Fz_unload) > 10:
    z_unload = np.polyfit(Fz_unload, P_unload, 2)
    p_unload = np.poly1d(z_unload)
    fz_range_unload = np.linspace(Fz_unload.min(), Fz_unload.max(), 100)
    ax.plot(fz_range_unload, p_unload(fz_range_unload), '--', color=COLORS[3], linewidth=2, alpha=0.7)

ax.set_xlabel('Normal Force |Fz| [N]')
ax.set_ylabel('Mean Pressure [hPa]')
ax.set_title('Normal Force Response: Loading vs Unloading')
ax.legend()
ax.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "03_normal_hysteresis.png"), dpi=300, bbox_inches='tight')
plt.close()

if len(Fz_unload) > 0 and len(Fz_load) > 0:
    hysteresis = np.abs(np.mean(P_load) - np.mean(P_unload))
    print(f"Average hysteresis: {hysteresis:.4f} hPa")

Average hysteresis: 0.2107 hPa


## 2) Shear response and directionality
Dataset: constant Fz levels and shear sweeps in +x, −x, +y, −y.

Set `left/right/top/bottom` mapping to match the PCB.

In [4]:
# ----------------- 2) SHEAR RESPONSE & DIRECTIONALITY -----------------
DATA_PATH = rf"C:\Users\aurir\OneDrive - epfl.ch\Thesis- Biorobotics Lab\train_validation_test_data\train_data_v{SENSOR_VERSION}2.csv"

df = load_data(DATA_PATH)

Fx = df["fx"].to_numpy()
Fy = df["fy"].to_numpy()
Fz = df["fz"].to_numpy()

# Keep contact samples
contact = np.abs(Fz) > 0.5

# ---- Figure: Each barometer vs Fx (shear in x direction) ----
fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharex=True)
axes = axes.flatten()

for i, col in enumerate(BARO_COLS):
    ax = axes[i]
    baro = df[col].to_numpy()
    
    # Scatter plot
    ax.scatter(Fx[contact], baro[contact], s=4, alpha=0.4, color='#008780')
    
    # Fit line
    mask = np.isfinite(Fx) & np.isfinite(baro) & contact
    if mask.sum() >= 50:
        coeffs = np.polyfit(Fx[mask], baro[mask], 1)
        slope, intercept = coeffs[0], coeffs[1]
        fx_range = np.array([Fx[contact].min(), Fx[contact].max()])
        ax.plot(fx_range, slope * fx_range + intercept, 
               linewidth=2, color='#44b155', label=f'slope={slope:.2f} hPa/N')
        ax.legend(fontsize=8)
    
    ax.set_ylabel(f'{col} [hPa]', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_title(f'{col}', fontsize=10)
    
    if i >= 3:  # Bottom row
        ax.set_xlabel('Fx [N]', fontsize=10)

fig.suptitle('Barometer Response to Shear Force (Fx)', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "04_shear_response_Fx.png"), dpi=300, bbox_inches='tight')
plt.close()

# ---- Figure: Each barometer vs Fy (shear in y direction) ----
fig, axes = plt.subplots(2, 3, figsize=(12, 7), sharex=True)
axes = axes.flatten()

for i, col in enumerate(BARO_COLS):
    ax = axes[i]
    baro = df[col].to_numpy()
    
    # Scatter plot
    ax.scatter(Fy[contact], baro[contact], s=4, alpha=0.4, color='#008780')
    
    # Fit line
    mask = np.isfinite(Fy) & np.isfinite(baro) & contact
    if mask.sum() >= 50:
        coeffs = np.polyfit(Fy[mask], baro[mask], 1)
        slope, intercept = coeffs[0], coeffs[1]
        fy_range = np.array([Fy[contact].min(), Fy[contact].max()])
        ax.plot(fy_range, slope * fy_range + intercept, 
               linewidth=2, color='#44b155', label=f'slope={slope:.2f} hPa/N')
        ax.legend(fontsize=8)
    
    ax.set_ylabel(f'{col} [hPa]', fontsize=10)
    ax.grid(True, alpha=0.3)
    ax.set_title(f'{col}', fontsize=10)
    
    if i >= 3:  # Bottom row
        ax.set_xlabel('Fy [N]', fontsize=10)

fig.suptitle('Barometer Response to Shear Force (Fy)', fontsize=13, fontweight='bold')
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "05_shear_response_Fy.png"), dpi=300, bbox_inches='tight')
plt.close()

# Print sensitivity for each barometer
print("\n=== Barometer Sensitivity to Shear Forces ===")
print("\nFx sensitivity (hPa/N):")
for col in BARO_COLS:
    baro = df[col].to_numpy()
    mask = np.isfinite(Fx) & np.isfinite(baro) & contact
    if mask.sum() >= 50:
        slope = np.polyfit(Fx[mask], baro[mask], 1)[0]
        print(f"  {col}: {slope:7.3f}")

print("\nFy sensitivity (hPa/N):")
for col in BARO_COLS:
    baro = df[col].to_numpy()
    mask = np.isfinite(Fy) & np.isfinite(baro) & contact
    if mask.sum() >= 50:
        slope = np.polyfit(Fy[mask], baro[mask], 1)[0]
        print(f"  {col}: {slope:7.3f}")


=== Barometer Sensitivity to Shear Forces ===

Fx sensitivity (hPa/N):
  b1:   0.070
  b2:  -0.081
  b3:   0.106
  b4:   0.245
  b5:   0.412
  b6:   0.130

Fy sensitivity (hPa/N):
  b1:   0.178
  b2:   0.010
  b3:  -0.049
  b4:  -0.306
  b5:  -0.315
  b6:  -0.115


## 3) Contact localization
Dataset: taps on a known grid at fixed Fz (and optionally multiple force levels).

Includes baseline linear regression + optional pressure centroid (needs barometer coordinates).

In [5]:
# ----------------- 3) CONTACT LOCALIZATION -----------------
DATA_PATH = rf"C:\Users\aurir\OneDrive - epfl.ch\Thesis- Biorobotics Lab\train_validation_test_data\train_data_v{SENSOR_VERSION}3.csv"

df = load_data(DATA_PATH)

contact = np.abs(df["fz"].to_numpy()) > 0.5

X = df.loc[contact, BARO_COLS].to_numpy()
x_true = df.loc[contact, "x"].to_numpy()
y_true = df.loc[contact, "y"].to_numpy()

# ---- Baseline: linear regression (closed form) ----
X1 = np.c_[np.ones(len(X)), X]
wx, *_ = np.linalg.lstsq(X1, x_true, rcond=None)
wy, *_ = np.linalg.lstsq(X1, y_true, rcond=None)
x_hat = X1 @ wx
y_hat = X1 @ wy

# ---- Figure: true vs estimated (scatter) ----
plt.figure(figsize=(5,5))
plt.scatter(x_true, y_true, s=4, alpha=0.35, label="true", color=COLORS[0])
plt.scatter(x_hat, y_hat, s=4, alpha=0.35, label="LR estimate", color=COLORS[4])
plt.xlabel("x [mm]")
plt.ylabel("y [mm]")
plt.title("Contact localization: true vs LR estimate")
plt.axis("equal")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "06_localization_scatter.png"), dpi=300, bbox_inches='tight')
plt.close()

# ---- Figure: error heatmap (binned) ----
err = np.sqrt((x_hat-x_true)**2 + (y_hat-y_true)**2)

nbx, nby = 20, 10
xb = np.linspace(np.nanmin(x_true), np.nanmax(x_true), nbx+1)
yb = np.linspace(np.nanmin(y_true), np.nanmax(y_true), nby+1)

H = np.full((nby, nbx), np.nan)
for i in range(nbx):
    for j in range(nby):
        m = (x_true>=xb[i]) & (x_true<xb[i+1]) & (y_true>=yb[j]) & (y_true<yb[j+1])
        if m.sum() >= 20:
            H[j,i] = np.nanmean(err[m])

plt.figure(figsize=(6.5,3.2))
plt.imshow(H, origin="lower", aspect="auto",
           extent=[xb[0], xb[-1], yb[0], yb[-1]], cmap=CMAP_THEME)
plt.colorbar(label="Mean localization error [mm]")
plt.xlabel("x [mm]")
plt.ylabel("y [mm]")
plt.title("Localization error heatmap (LR baseline)")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "07_localization_error_heatmap.png"), dpi=300, bbox_inches='tight')
plt.close()

print("LR localization RMSE [mm]:", np.sqrt(np.nanmean(err**2)))

# ---- Figure: error vs force level (if multiple levels exist) ----
Fz = -df.loc[contact, "fz"].to_numpy()  # positive compression
bins = [0, 2, 4, 6, 8, 10, 20]
centers, means = [], []
for a,b in zip(bins[:-1], bins[1:]):
    m = (Fz>=a) & (Fz<b)
    if m.sum() < 50:
        continue
    centers.append((a+b)/2)
    means.append(np.nanmean(err[m]))

plt.figure(figsize=(5.5,3.5))
plt.plot(centers, means, marker="o", color=COLORS[1], linewidth=1.5, markersize=6)
plt.xlabel("Fz bin center [N]")
plt.ylabel("Mean localization error [mm]")
plt.title("Localization error vs force level")
plt.grid(True)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "08_localization_vs_force.png"), dpi=300, bbox_inches='tight')
plt.close()

LR localization RMSE [mm]: 6.492314905328997


## 4) Noise floor and resolution
Dataset: ~60 s no-contact then ~60 s constant contact at fixed Fz.

In [6]:
# ----------------- 4) NOISE FLOOR & RESOLUTION -----------------
DATA_PATH = rf"C:\Users\aurir\OneDrive - epfl.ch\Thesis- Biorobotics Lab\train_validation_test_data\train_data_v{SENSOR_VERSION}4.csv"

df = load_data(DATA_PATH)

t = df["t_s"].to_numpy()
P = df[BARO_COLS].to_numpy()
Fz = df["fz"].to_numpy()

no_contact = np.abs(Fz) < 0.2
contact = np.abs(Fz) > 1.0

# ---- Figure: barometer signals over time ----
plt.figure(figsize=(7,3.5))
for i, c in enumerate(BARO_COLS):
    plt.plot(t, df[c].to_numpy(), linewidth=0.8, label=c, alpha=0.9, color=COLORS[i])
plt.xlabel("time [s]")
plt.ylabel("pressure [hPa]")
plt.ylim(-2, 2)
plt.title("Barometers over time (no-contact then contact)")
plt.grid(True)
plt.legend(ncol=3)
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "09_noise_time_series.png"), dpi=300, bbox_inches='tight')
plt.close()

# ---- Figure: std per channel ----
std_nc = np.nanstd(P[no_contact,:], axis=0)
std_c  = np.nanstd(P[contact,:], axis=0)

x = np.arange(len(BARO_COLS))
plt.figure(figsize=(6,3.5))
plt.bar(x-0.2, std_nc, width=0.4, label="no-contact", color=COLORS[0])
plt.bar(x+0.2, std_c,  width=0.4, label="constant contact", color=COLORS[4])
plt.xticks(x, BARO_COLS)
plt.ylabel("Std dev [hPa]")
plt.title("Noise per channel")
plt.grid(True, axis="y")
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "10_noise_per_channel.png"), dpi=300, bbox_inches='tight')
plt.close()

# ---- Minimum detectable force (1-sigma) using your measured sensitivity ----
S_hPa_per_N = 1.0  # set from Section 1 (low-force slope), e.g. 2.3
mean_std_hPa = float(np.nanmean(std_nc))
f_min = mean_std_hPa / max(S_hPa_per_N, 1e-9)

print("Mean no-contact std (hPa):", mean_std_hPa)
print("Assumed sensitivity (hPa/N):", S_hPa_per_N)
print("Estimated min detectable force (1-sigma) [N]:", f_min)

c:\Users\aurir\anaconda3\Lib\site-packages\numpy\lib\nanfunctions.py:1879: RuntimeWarning: Degrees of freedom <= 0 for slice.
  var = nanvar(a, axis=axis, dtype=dtype, out=out, ddof=ddof,


Mean no-contact std (hPa): nan
Assumed sensitivity (hPa/N): 1.0
Estimated min detectable force (1-sigma) [N]: nan


C:\Users\aurir\AppData\Local\Temp\ipykernel_14408\303205020.py:46: RuntimeWarning: Mean of empty slice
  mean_std_hPa = float(np.nanmean(std_nc))


## 5) Drift and temperature sensitivity
Dataset: ~10 min no-contact then ~10 min constant load.

Temperature plot runs only if columns like `t1..t6` exist.

In [7]:
# ----------------- 5) DRIFT & TEMPERATURE SENSITIVITY -----------------
DATA_PATH = rf"C:\Users\aurir\OneDrive - epfl.ch\Thesis- Biorobotics Lab\train_validation_test_data\train_data_v{SENSOR_VERSION}5.csv"

df = load_data(DATA_PATH)

t = df["t_s"].to_numpy()
Praw = df[BARO_COLS].to_numpy()

alpha = 1e-4
Pema = np.zeros_like(Praw)
Pema[0] = Praw[0]
for i in range(1, len(Praw)):
    Pema[i] = alpha * Praw[i] + (1 - alpha) * Pema[i-1]
Pcorr = Praw - Pema

mpr = Praw.mean(axis=1)
mpc = Pcorr.mean(axis=1)

# ---- Figure: drift before vs after EMA ----
plt.figure(figsize=(7,3.5))
plt.plot(t, mpr, label="raw mean pressure", linewidth=1.0, color=COLORS[2])
plt.plot(t, mpc, label="EMA-corrected mean pressure", linewidth=1.0, color=COLORS[1])
plt.xlabel("time [s]")
plt.ylabel("pressure [hPa]")
plt.title("Baseline drift before vs after EMA removal")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "11_drift_correction.png"), dpi=300, bbox_inches='tight')
plt.close()

# ---- Figure: pressure vs temperature (if available) ----
temp_cols = [c for c in df.columns if re.match(r"^t\d+$", str(c))]
if len(temp_cols) >= 1:
    T = df[temp_cols].mean(axis=1).to_numpy()
    plt.figure(figsize=(5.5,4))
    plt.scatter(T, mpr, s=4, alpha=0.35, color=COLORS[3])
    m = np.isfinite(T) & np.isfinite(mpr)
    if m.sum() > 50:
        a,b = np.polyfit(T[m], mpr[m], 1)
        xline = np.linspace(np.nanmin(T), np.nanmax(T), 100)
        plt.plot(xline, a*xline + b, linewidth=2, color=COLORS[0])
        print("dP/dT (raw mean) =", a, "hPa/deg")
    plt.xlabel("temperature [deg]")
    plt.ylabel("mean pressure [hPa]")
    plt.title("Pressure vs temperature")
    plt.grid(True)
    plt.tight_layout()
    plt.savefig(os.path.join(OUTPUT_DIR, "12_temperature_sensitivity.png"), dpi=300, bbox_inches='tight')
    plt.close()
else:
    print("No temperature columns found (expected t1..t6).")

No temperature columns found (expected t1..t6).


## 6) Version or material comparison
Two datasets: same normal-force protocol on version/material A vs B.

In [8]:
# ----------------- 6) VERSION / MATERIAL COMPARISON -----------------
DATA_PATH_A = r"C:\Users\aurir\OneDrive - epfl.ch\Thesis- Biorobotics Lab\train_validation_test_data\normal_v5.1.csv"
DATA_PATH_B = r"C:\Users\aurir\OneDrive - epfl.ch\Thesis- Biorobotics Lab\train_validation_test_data\normal_v5.02.csv"

dfa = load_data(DATA_PATH_A)
dfb = load_data(DATA_PATH_B)

Pa = moving_average(mean_pressure(dfa), win=5)
Pb = moving_average(mean_pressure(dfb), win=5)
Fza = -moving_average(dfa["fz"].to_numpy(), win=5)
Fzb = -moving_average(dfb["fz"].to_numpy(), win=5)

sega = segment_runs_by_contact(dfa, fz_col="fz", contact_thr=0.2, min_len=300)
segb = segment_runs_by_contact(dfb, fz_col="fz", contact_thr=0.2, min_len=300)
sa,ea = sega[0] if len(sega) else (0,len(dfa))
sb,eb = segb[0] if len(segb) else (0,len(dfb))

# ---- Figure: mean pressure vs Fz comparison ----
plt.figure(figsize=(5.5,4))
plt.plot(Fza[sa:ea], Pa[sa:ea], label="A", alpha=0.9, color=COLORS[2], linewidth=1.5)
plt.plot(Fzb[sb:eb], Pb[sb:eb], label="B", alpha=0.9, color=COLORS[3], linewidth=1.5)
plt.xlabel("Fz [N] (positive = compression)")
plt.ylabel("Mean pressure [hPa]")
plt.title("Normal response comparison")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "13_version_comparison.png"), dpi=300, bbox_inches='tight')
plt.close()

# ---- Figure: quick hysteresis metric ----
def hysteresis_metric(F, P):
    m = np.isfinite(F) & np.isfinite(P)
    F = F[m]; P = P[m]
    if len(F) < 50:
        return np.nan
    area = np.trapz(P, F)  # integral P dF
    span = (np.nanmax(F)-np.nanmin(F)) * (np.nanmax(P)-np.nanmin(P)) + 1e-9
    return np.abs(area)/span

hA = hysteresis_metric(Fza[sa:ea], Pa[sa:ea])
hB = hysteresis_metric(Fzb[sb:eb], Pb[sb:eb])

plt.figure(figsize=(4.5,3.5))
plt.bar([0,1], [hA, hB], color=[COLORS[2], COLORS[3]])
plt.xticks([0,1], ["A","B"])
plt.ylabel("Normalized hysteresis (area/span)")
plt.title("Hysteresis comparison (quick metric)")
plt.grid(True, axis="y")
plt.tight_layout()
plt.savefig(os.path.join(OUTPUT_DIR, "14_hysteresis_comparison.png"), dpi=300, bbox_inches='tight')
plt.close()

print("Hysteresis metric A:", hA)
print("Hysteresis metric B:", hB)

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\aurir\\OneDrive - epfl.ch\\Thesis- Biorobotics Lab\\train_validation_test_data\\normal_v5.1.csv'